## Multiprocessing 

In [1]:
import multiprocessing as mp
print("number of cpu :", mp.cpu_count())

number of cpu : 8


In [2]:
def printing(data = 'asia'):
    print(data)

###  Creating a single process parallelly

1. Need to used start to start the process 
2. Need to use join to wait for the program to stoop and when its done, close the program otherwise need to kill with the task manager

In [3]:
proc = mp.Process(target = printing)
proc.start()
proc.join()

asia


### Creating a multiple process all printing differnet names 

1. The following implementation may not work parallely as it waits afterward for ending of one process .. and than going to another loop

In [4]:
names = ['asia', 'africa', 'india', 'antarctica']
for name in names:
    proc = mp.Process(target = printing, args=(name,))
    proc.start();
    proc.join()

asia
africa
india
antarctica


2. The following implementaion may work as the process are waited for after all the process have started 

In [5]:
names = ['asia', 'africa', 'india', 'antarctica']
prolist = []
for name in names:
    proc = mp.Process(target = printing, args=(name,))
    proc.start();
    prolist.append(proc)
    
for proc in prolist:
    proc.join()

asia
africa
india
antarctica


## For better demonstartion 
1. Below are some interating functions which are created for giving insights on more about the same problem as mentioned 

In [6]:
def printing(fixed = 'my', data = 5):
    for i in range(data): print(fixed, i)

In [7]:
for i in range(4):
    proc = mp.Process(target = printing, args = ('data'+str(i),5,))
    proc.start()
    proc.join()


data0 0
data0 1
data0 2
data0 3
data0 4
data1 0
data1 1
data1 2
data1 3
data1 4
data2 0
data2 1
data2 2
data2 3
data2 4
data3 0
data3 1
data3 2
data3 3
data3 4


2. Below is the correct implementation of process which are working multiple program

In [8]:
prolist = []
for i in range(4):
    proc = mp.Process(target = printing, args = ('data'+str(i),5,))
    prolist.append(proc)
    proc.start()
    
for proc in prolist:
    proc.join()

data0 0
data0 1
data0 2
data1 0
data0 3
data1 1
data0 4
data2 0
data1 2
data2 1
data3 0
data2 2
data3 1
data1 3
data2 3
data1 4
data3 2
data2 4
data3 3
data3 4


## Queue in Multiprocessing

->this queue is used by multiprocessing library which keeps on waiting for get() until something is not being put() --> wheare as in normal queue(), this thing is not possible, it will simply say that the queue is empty.

Wheaas same thing could be replicated like that of normal queue by queue().get_nowait()

In [9]:
import queue as Q
queue = mp.Queue()
for i in range(5):
    queue.put(i)
while not queue.empty():
    print(queue.get())
while(True):
    try:
        print(queue.get_nowait())
    except Q.Empty:  #this acts like queue is ended 
        break

### Lock

--> if there are multipule process using same segment of code, then one can use lock so than only the one who aquired the lock can access the code.

--> Remaining can only access the code only when realeased

acquire() and release()

In [10]:
try:
    print("nation")
except KeyboardInterrupt:
    print("Intrupt")
else:
    print(" Else Statement Executed ")   # else statement occurs
    #if there is no exception is found in try

nation
 Else Statement Executed 


##### All the child process will get different process id if we include line 14 as there is a time sleep. Due to which the pc will shift its focus to another process meanwhile

In [23]:
import queue as Q
import time
def task_scheduler(done_process, to_be_done_process):
    while True:
        #will run until the process list is not empty
        try:
            task = to_be_done_process.get_nowait()
        except Q.Empty:
            break
        else:
            # will be executed if there is no exception 
            print(task, mp.current_process())
            done_process.put(task)
            time.sleep(0.5) #if there is a time --> then the process will shift to another
            #since there is a waiting time --> theory confirmed

In [22]:
done_process = mp.Queue()
to_be_done_process = mp.Queue()

for i in range(10):
    to_be_done_process.put(i)
    
processes = []
for i in range(10):
    proc = mp.Process(target= task_scheduler, args=(done_process, to_be_done_process))
    processes.append(proc)
    proc.start()
    
for proc in processes:
    proc.join()

while True:
    try:
        print(" executed process {}".format(done_process.get_nowait()))
    except Q.Empty:
        break

0 <Process(Process-58, started)>
1 <Process(Process-59, started)>
2 <Process(Process-60, started)>
3 <Process(Process-61, started)>
4 <Process(Process-62, started)>
5 <Process(Process-63, started)>
6 <Process(Process-64, started)>
7 <Process(Process-65, started)>
8 <Process(Process-66, started)>
9 <Process(Process-67, started)>
 executed process 0
 executed process 1
 executed process 2
 executed process 3
 executed process 4
 executed process 5
 executed process 6
 executed process 7
 executed process 8
 executed process 9


## Pooling Process
Basically dividing each of the process to multiple pools --> load balancing 
* the same is used for creating a pool of parallel process such that it keeps on accepting the argument till the list is not over.

In [207]:
work = [['A',2],['B',3],['C',1],['D',2],['F',1],['G',1]]

def workdone(working_data):
    print('working for {} for timming {}'.format(working_data[0],working_data[1]))
    print(mp.current_process())
    time.sleep(working_data[1])
    print(' Process finished {} with current id {}'.format(working_data[0], mp.current_process()))
    return mp.current_process().pid

In [208]:
def pool_handler():
    p = mp.Pool(3)  #means 2 process will be executing parallely
    h = p.map(workdone, work)  #maping workdone function to argument work
    print(list(h))
pool_handler()

working for B for timming 3
working for A for timming 2
working for C for timming 1
<ForkProcess(ForkPoolWorker-1174, started daemon)>
<ForkProcess(ForkPoolWorker-1176, started daemon)>
<ForkProcess(ForkPoolWorker-1175, started daemon)>
 Process finished C with current id <ForkProcess(ForkPoolWorker-1176, started daemon)>
working for D for timming 2
<ForkProcess(ForkPoolWorker-1176, started daemon)>
 Process finished A with current id <ForkProcess(ForkPoolWorker-1174, started daemon)>
working for F for timming 1
<ForkProcess(ForkPoolWorker-1174, started daemon)>
 Process finished B with current id <ForkProcess(ForkPoolWorker-1175, started daemon)>
working for G for timming 1
<ForkProcess(ForkPoolWorker-1175, started daemon)>
 Process finished F with current id <ForkProcess(ForkPoolWorker-1174, started daemon)>
 Process finished D with current id <ForkProcess(ForkPoolWorker-1176, started daemon)>
 Process finished G with current id <ForkProcess(ForkPoolWorker-1175, started daemon)>
[265

Process ForkPoolWorker-1175:
Process ForkPoolWorker-1174:
Process ForkPoolWorker-1176:
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/oxygen/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worke

### Testing if the same can be used with gym 

In [192]:
import gym
gym.logger.set_level(40)
# from collections import namedtuple
import multiprocessing as mp

data_storage = mp.Queue()
# exp = namedtuple('Experience',['process_id','next_state','reward'])


In [193]:
def worker(data_af):
    data = []
    exp = namedtuple('Experience',['process_id'])
    print("Process started for pid ",mp.current_process().pid)
    env = gym.make('CartPole-v0')
    state = env.reset()
    while True:
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
#         data.append(exp(mp.current_process(), next_state, reward))
        data.append([mp.current_process().pid,next_state,reward])
        state = next_state
        if done:
            break
    data_af.put(data) 

In [194]:
processes = []
nos = 20
for i in range(nos):
    proc = mp.Process(target=worker, args=(data_storage,))
    processes.append(proc)
    proc.start()
    
for proc in processes:
    proc.join()
    
while not data_storage.empty():
    g  = data_storage.get_nowait()
    print(len(g))
    print()
    print(g)
    print()


Process started for pid  26255
Process started for pid  26258
Process started for pid  26259
Process started for pid  26264
Process started for pid  26267
Process started for pid  26270
Process started for pid  26277
Process started for pid  26273
Process started for pid  26283
Process started for pid  26286
Process started for pid  26289
Process started for pid  26298
Process started for pid  26292
Process started for pid  26294
Process started for pid  26306
Process started for pid  26307
Process started for pid  26316
Process started for pid  26311
Process started for pid  26314
Process started for pid  26324
8

[[26255, array([ 0.02186186, -0.23642184,  0.04666768,  0.34097024]), 1.0], [26255, array([ 0.01713342, -0.43217564,  0.05348708,  0.64799657]), 1.0], [26255, array([ 0.00848991, -0.62800036,  0.06644701,  0.95703128]), 1.0], [26255, array([-0.0040701 , -0.82394991,  0.08558764,  1.26982803]), 1.0], [26255, array([-0.0205491 , -1.020054  ,  0.1109842 ,  1.58803949]), 1.0], [

## Multiple rendering using multiple process

In [2]:
import gym
gym.logger.set_level(40)
# from collections import namedtuple
import multiprocessing as mp
# exp = namedtuple('Experience',['process_id','next_state','reward'])

In [13]:
data_storage = mp.Queue()
def worker(data_af):
    data = []
    print("Process started for pid ",mp.current_process().pid)
    env = gym.make('CartPole-v0')
    for i in range(10):
        state = env.reset()
        while True:
            env.render()
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
    #         data.append(exp(mp.current_process(), next_state, reward))
            data.append([mp.current_process().pid,next_state,reward])
            state = next_state
            if done:
                break
        print('\n {} length \n'.format(len(data)))
    env.close()
    data_af.put(data)

In [14]:
processes = []
nos = 1
for i in range(nos):
    proc = mp.Process(target=worker, args=(data_storage,))
    processes.append(proc)
    proc.start()
    
while not data_storage.empty():
    g  = data_storage.get_nowait()
    print(len(g))
    print()
    print(g)
    print()
    
for proc in processes:
    proc.join()

Process started for pid  30468

 13 length 


 30 length 


 41 length 


 53 length 


 79 length 


 130 length 


 172 length 


 198 length 


 213 length 


 229 length 



* Data is not being passed from the queue as there is limit in queue ( the amount of data that can be stored in a queue at a time is fixed).
* Thus before we can add the data, data should be removed so that function can be stored properly
* Global datas are not acessible by the every child and parent. if any modifiction made by the child process may not be accessibe to parent process
* We can use shared memeory such as queue, pipe, and manager.

## Communication using Manager 

* `Multiprocessing`, Whenever process starts, another set of process startes namely Manger process.
* Any new process which parent process wants to create can be done by communicating with the manager.
* The data which is stored and maintained by the manager is accessible by remaining remainied set of process.
* Manger is not an effective way of communicaton
* Thus for this `Queue` and `pipe` is always an effective way of communicaiton

In [128]:
## testing manager
import multiprocessing as mp

def printing(data):
    print(' Printing {} by child process {}'.format(data,mp.current_process().pid))
    
def insert_record(record, records):
    records.append(record)
    
with mp.Manager() as manager:
    record = ['general',11]
    records = manager.list([('Sam', 10), ('Adam', 9), ('Kevin',9)]) 
    p1 = mp.Process(target = insert_record, args=(record, records,))
    p2 = mp.Process(target = printing, args=(record,))
    p1.start()
    p1.join()
    p2.start()
    p2.join()
    print(records)
    

 Printing ['general', 11] by child process 5409
[('Sam', 10), ('Adam', 9), ('Kevin', 9), ['general', 11]]


### Creating Multiple instance of gym environment parallely and storing in another process and making data globally available 

In [1]:
import multiprocessing as mp
from collections import namedtuple

def worker(data_af):
    print("Process started for pid ",mp.current_process().pid)
    env = gym.make('CartPole-v0')
    for i in range(10):
        state = env.reset()
        while True:
            env.render()
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
    #         data.append(exp(mp.current_process(), next_state, reward))
            data_af.append([mp.current_process().pid,state,next_state,reward,action])
            state = next_state
            if done:
                break
    env.close()

In [3]:
nos = 10
process = []
data = [] 
exp = namedtuple('Experience',['pid','state','next_state','reward','action'])
with mp.Manager() as manager:
    records = manager.list([]) 
    for i in range(nos):
        proc = mp.Process(target = worker, args=(records,))
        process.append(proc)
        proc.start()
    
    for proc in process:
        proc.join()

    print(len(records))
    data = [exp(x[0],x[1],x[2],x[3],x[4]) for x in records]

Process started for pid  4544
Process started for pid  4546
Process started for pid  4558
Process started for pid  4550
Process started for pid  4554
Process started for pid  4566
Process started for pid  4562
Process started for pid  4578
Process started for pid  4570
Process started for pid  4574
2076


## Pipes 

In [4]:
import multiprocessing as mp

In [65]:
import multiprocessing as mp
from collections import namedtuple

def worker(env,child):
    print("Process started for pid ",mp.current_process().pid)
#     exp = namedtuple('experience',['state','next_state','reward','action','done'])
    for i in range(10):
        state = env.reset()
        while True:
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
            child.send([state, next_state, reward, action, done])
            state = next_state
            if done:
                env.close()
                break
    env.close()

In [66]:
parent,child = mp.Pipe()
env = gym.make('CartPole-v0')
proc = mp.Process(target = worker, args = (env,child,))
proc.start()
for i in range(10):
    while True:
        data = parent.recv()
        print(data)
        if data[-1] is True:
            break
proc.join()

Process started for pid  9729
[array([0.04509514, 0.04197616, 0.00904608, 0.02106077]), array([ 0.04593466, -0.15327434,  0.0094673 ,  0.31658406]), 1.0, 0, False]
[array([ 0.04593466, -0.15327434,  0.0094673 ,  0.31658406]), array([0.04286918, 0.04171149, 0.01579898, 0.02690176]), 1.0, 1, False]
[array([0.04286918, 0.04171149, 0.01579898, 0.02690176]), array([ 0.04370341, -0.15363342,  0.01633702,  0.32452727]), 1.0, 0, False]
[array([ 0.04370341, -0.15363342,  0.01633702,  0.32452727]), array([0.04063074, 0.04125214, 0.02282756, 0.0370408 ]), 1.0, 1, False]
[array([0.04063074, 0.04125214, 0.02282756, 0.0370408 ]), array([ 0.04145578, -0.15418959,  0.02356838,  0.33683774]), 1.0, 0, False]
[array([ 0.04145578, -0.15418959,  0.02356838,  0.33683774]), array([0.03837199, 0.04058917, 0.03030513, 0.0516792 ]), 1.0, 1, False]
[array([0.03837199, 0.04058917, 0.03030513, 0.0516792 ]), array([ 0.03918377,  0.23526377,  0.03133872, -0.23129027]), 1.0, 1, False]
[array([ 0.03918377,  0.23526377

### Process to run multiple processes and collect data in one part

In [67]:
def sync(parent,data):
    for i in range(10):
    while True:
        data = parent.recv()
        print(data)
        if data[-1] is True:
            break
proc.join()

SyntaxError: unexpected EOF while parsing (<ipython-input-67-8f32e6ed9252>, line 1)

In [62]:
nos = 10
process = []
parent_collection = []
main_data = [] 
for i in range(nos):
    parent,child = mp.Pipe()
    env = gym.make('CartPole-v0')
    proc = mp.Process(target = worker, args = (env,child,))
    process.append(proc)
    parent_collection.append(parent)

with mp.Manager() as manager:
    data = manager.list([])
    proc = mp.Process(target = sync, args=(parent,data,))
    proc.start()
    proc.join()
    main_data = [x for x in data]
    
for i in range(10):
    while True:
        data = parent.recv()
        print(data)
        if data[-1] is True:
            break
proc.join()

Process started for pid  9075
[array([-0.00407652,  0.04051155,  0.01150213, -0.03399888]), array([-0.00326629, -0.15477344,  0.01082215,  0.26229078]), 1.0, 0, False]
[array([-0.00326629, -0.15477344,  0.01082215,  0.26229078]), array([-0.00636176, -0.35004819,  0.01606797,  0.5583674 ]), 1.0, 0, False]
[array([-0.00636176, -0.35004819,  0.01606797,  0.5583674 ]), array([-0.01336272, -0.54539196,  0.02723532,  0.85606901]), 1.0, 0, False]
[array([-0.01336272, -0.54539196,  0.02723532,  0.85606901]), array([-0.02427056, -0.35065151,  0.0443567 ,  0.57207286]), 1.0, 1, False]
[array([-0.02427056, -0.35065151,  0.0443567 ,  0.57207286]), array([-0.03128359, -0.15617868,  0.05579815,  0.29368714]), 1.0, 1, False]
[array([-0.03128359, -0.15617868,  0.05579815,  0.29368714]), array([-0.03440716, -0.35204993,  0.0616719 ,  0.6034329 ]), 1.0, 0, False]
[array([-0.03440716, -0.35204993,  0.0616719 ,  0.6034329 ]), array([-0.04144816, -0.54797776,  0.07374055,  0.91488596]), 1.0, 0, False]
[arr

In [44]:
data[-1]

True

In [29]:
a.send(exp('data'))

In [30]:
b.recv()

exp(name='data')

In [32]:
dir(a)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bad_message_length',
 '_check_closed',
 '_check_readable',
 '_check_writable',
 '_close',
 '_handle',
 '_poll',
 '_read',
 '_readable',
 '_recv',
 '_recv_bytes',
 '_send',
 '_send_bytes',
 '_writable',
 '_write',
 'close',
 'closed',
 'fileno',
 'poll',
 'readable',
 'recv',
 'recv_bytes',
 'recv_bytes_into',
 'send',
 'send_bytes',
 'writable']